In [ ]:

import requests
import json
import time

import csv
import time
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from geopy.extra.rate_limiter import RateLimiter
import logging
import re

In [2]:
state_route_patterns = {
    'AL': ['AL'],
    'AK': ['AK'],
    'AZ': ['AZ'],
    'AR': ['AR'],
    'CA': ['CA'],
    'CO': ['CO'],
    'CT': ['CT'],
    'DE': ['DE'],
    'FL': ['FL'],
    'GA': ['GA'],
    'HI': ['HI'],
    'ID': ['ID'],
    'IL': ['IL'],
    'IN': ['IN'],
    'IA': ['IA'],
    'KS': ['K'],
    'KY': ['KY'],
    'LA': ['LA'],
    'ME': ['ME'],
    'MD': ['MD'],
    'MA': ['MA'],
    'MI': ['M'],
    'MN': ['MN'],
    'MS': ['MS'],
    'MO': ['MO'],
    'MT': ['MT'],
    'NE': ['NE'],
    'NV': ['NV'],
    'NH': ['NH'],
    'NJ': ['NJ'],
    'NM': ['NM'],
    'NY': ['NY'],
    'NC': ['NC'],
    'ND': ['ND'],
    'OH': ['OH'],
    'OK': ['OK'],
    'OR': ['OR'],
    'PA': ['PA'],
    'RI': ['RI'],
    'SC': ['SC'],
    'SD': ['SD'],
    'TN': ['TN'],
    'TX': ['TX'],
    'UT': ['UT'],
    'VT': ['VT'],
    'VA': ['VA'],
    'WA': ['WA'],
    'WV': ['WV'],
    'WI': ['WI'],
    'WY': ['WY']
}

In [29]:
queries = {
    "exit":"""
        [out:json][timeout:25];
        // Define the area of interest
        nwr(id:{})->.citySearch;


        // Search for the road by name or ref
        way["ref"~"(;|^){}(;|$)"](around.citySearch:15000)->.road;

        // Extract nodes of type 'motorway_junction' on the road
        node(w.road)["highway"="motorway_junction"]["ref"~"^{}([a-zA-Z]|$)"]->.exits;

        // Output all exit nodes with their coordinates
        (.exits;); /*added by auto repair*/ (._;>;); /*end of auto repair*/ out body 1; 
    """,
    "junction":"""
        [out:json][timeout:25];

        nr(id:{})->.citySearch;

        way["ref"~"(;|^){}(;|$)"](around.citySearch:15000)->.w1;
        way["ref"~"(;|^){}(;|$)"](around.citySearch:15000)->.w2;

        // Get nodes that are part of w2
        node(w.w2)->.nodes_on_w2;
        node(w.w1)(w.w2) .on_both;
        // Get nodes that are within 100 meters of w2 but not part of w2
        node(w.w1)(around.w2:100)->.nodes_near_w2;
        (.nodes_near_w2; - .nodes_on_w2;)->.inter;


        // Output the result
        (.inter;.on_both;); out body 1;
        >;
        out skel qt; 
    """,
    "road_city":"""
        [out:json][timeout:25];
        // Define the area of interest
        nr(id:{})->.citySearch;

        // Search for the road by name or ref
        way["ref"~"(;|^){}(;|$)"](around.citySearch:15000)->.road;

        // Output all city nodes with their coordinates
        (.road;); /*added by auto repair*/ (._;>;); /*end of auto repair*/ out body 1; 
    """,
}

In [41]:
def parse_address(address):
    # Initialize variables
    roads_inter = []
    roads_along = []
    exit_number = None
    road_to_exit = None
    fall_back_road = None
    addr = address
    addr = addr.replace('/', ',')
    addr = addr.replace('&', ',')
    addr = addr.replace('-', ' ')
    
    
    parts = [part.strip() for part in addr.split(',')]

    exit_pattern = re.compile(r'^EXIT\s?(\d+(\s|)[A-Z]?)', re.IGNORECASE)
    
    for idx,part in enumerate(parts):
        
        exit_match = exit_pattern.match(part)
        if exit_match:
            return parts[idx-1], exit_match.group(1), roads_inter, roads_along
    
    addr = address
    addr = addr.replace('/', ',/,')
    addr = addr.replace('&', ',&,')
    addr = addr.replace('-', ' ')
    
    
    parts = [part.strip() for part in addr.split(',')]
    print(parts)
    fall_back_road = parts[0]
    
    for idx,part in enumerate(parts):
        
        if part == "&" and len(roads_inter) == 0:
            roads_inter.append(parts[idx-1])
            roads_inter.append(parts[idx+1])
        
        if part == "/" and len(roads_along) == 0:
            roads_along.append(parts[idx-1])
            roads_along.append(parts[idx+1])
            
    

    
    return road_to_exit, exit_number, roads_inter, fall_back_road
     

In [15]:
def get_location_from_overpass(key, id, state, *args):
    
    # Overpass API endpoint
    overpass_url = "http://overpass-api.de/api/interpreter"
    
    # Create the Overpass query
    state_pattern = state_route_patterns[state][0]
    if key == "exit":
        road = args[0]
        if args[0].startswith("SR"):
            road = args[0][2:]
            s = "(SH|SR|{})".format(state_pattern)
            road = s + road
        query = queries[key].format(id, road,args[1])
        
    elif key == "junction":
        road1 = args[2][0]
        road2 = args[2][1]
        if road1.startswith("SR"):
            road1 = road1[2:]
            s = "(SH|SR|{})".format(state_pattern)
            road1 = s + road1
        if road2.startswith("SR"):
            road2 = road2[2:]
            s = "(SH|SR|{})".format(state_pattern)
            road2 = s + road2
            
        query = queries[key].format(id, road1, road2)
    
    elif key == "road_city":
        road = args[3]
        if road.startswith("SR"):
            road = road[2:]
            s = "(SH|SR|{})".format(state_pattern)
            road = s + road
        query = queries[key].format(id, road)
        
    
    print(query)
    try:
        # Add delay to respect rate limits
        time.sleep(1)
        
        # Make the request
        response = requests.get(overpass_url, params={'data': query})
        response.raise_for_status()
        
        return response.json()
    
    except requests.exceptions.RequestException as e:
        logging.error(f"Error querying Overpass API: {str(e)}")
        return None
    


In [42]:
def get_query_type(road_to_exit, exit_number, roads_inter, fall_back_road):
    if exit_number is not None and road_to_exit is not None:
        # If we have an exit number and road, we need the exit query
        return "exit"
    elif len(roads_inter) == 2:
        # If we have two intersecting roads, we need the junction query
        return "junction"
    elif fall_back_road is not None:
        # If we only have a road name, use the road_city query
        return "road_city"
    else:
        return None
    
args = parse_address("I-94, EXIT 15 B")
print(args)


('I 94', '15 B', [], [])


In [38]:
def get_coordinates_from_csv(input_file):
    # Read the CSV file
    df = pd.read_csv(input_file)
    
    # Create empty lists to store coordinates
    coordinates = []

    
    # Iterate through each row
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        # Parse the address
        road_to_exit, exit_number, roads_inter, fall_back_road = parse_address(row['Address'])
        
        # Get query type
        query_type = get_query_type(road_to_exit, exit_number, roads_inter, fall_back_road)
        
        if query_type:
            # Get location data from Overpass
            result = get_location_from_overpass(
                query_type, 
                row['osm_id'], 
                row['State'],
                road_to_exit,
                exit_number,
                roads_inter,
                fall_back_road
            )
            
            # Extract coordinates if results exist
            if result and 'elements' in result and len(result['elements']) > 0:
                # Get first element's coordinates
                element = result['elements'][0]
                lat = element['lat'] if 'lat' in element else None
                lon = element['lon'] if 'lon' in element else None
            else:
                lat, lon = None, None
                
            # Add coordinates to original dataframe
            df.at[idx, 'latitude'] = lat
            df.at[idx, 'longitude'] = lon
            
            df.to_csv('checkpoint.csv', index=False)        
    
    return df

# Run the function
df = get_coordinates_from_csv('new_fuel.csv')

  0%|          | 0/8160 [00:00<?, ?it/s]


        [out:json][timeout:25];
        // Define the area of interest
        nwr(id:184155.0)->.citySearch;


        // Search for the road by name or ref
        way["ref"~"(;|^)I 44(;|$)"](around.citySearch:15000)->.road;

        // Extract nodes of type 'motorway_junction' on the road
        node(w.road)["highway"="motorway_junction"]["ref"~"^283([a-zA-Z]|$)"]->.exits;

        // Output all exit nodes with their coordinates
        (.exits;); /*added by auto repair*/ (._;>;); /*end of auto repair*/ out body 1; 
    


  0%|          | 1/8160 [00:03<7:00:29,  3.09s/it]


        [out:json][timeout:25];
        // Define the area of interest
        nwr(id:251430.0)->.citySearch;


        // Search for the road by name or ref
        way["ref"~"(;|^)I 94(;|$)"](around.citySearch:15000)->.road;

        // Extract nodes of type 'motorway_junction' on the road
        node(w.road)["highway"="motorway_junction"]["ref"~"^143([a-zA-Z]|$)"]->.exits;

        // Output all exit nodes with their coordinates
        (.exits;); /*added by auto repair*/ (._;>;); /*end of auto repair*/ out body 1; 
    


  0%|          | 2/8160 [00:06<6:54:49,  3.05s/it]

...

['US 41']

        [out:json][timeout:25];
        // Define the area of interest
        nr(id:1846532.0)->.citySearch;

        // Search for the road by name or ref
        way["ref"~"(;|^)US 41(;|$)"](around.citySearch:15000)->.road;

        // Output all city nodes with their coordinates
        (.road;); /*added by auto repair*/ (._;>;); /*end of auto repair*/ out body 1; 
    


  0%|          | 36/8160 [02:08<14:59:32,  6.64s/it]


        [out:json][timeout:25];
        // Define the area of interest
        nwr(id:1801527.0)->.citySearch;


        // Search for the road by name or ref
        way["ref"~"(;|^)I 70(;|$)"](around.citySearch:15000)->.road;

        // Extract nodes of type 'motorway_junction' on the road
        node(w.road)["highway"="motorway_junction"]["ref"~"^159([a-zA-Z]|$)"]->.exits;

        // Output all exit nodes with their coordinates
        (.exits;); /*added by auto repair*/ (._;>;); /*end of auto repair*/ out body 1; 
    


  0%|          | 36/8160 [02:16<8:31:36,  3.78s/it] 


OSError: [Errno 28] No space left on device